### GX Core Workflow - Using multiple data sources (Sqlite & Pandas)

- Import Required Libraries
- Create GX Context (Ephemeral)
- Create DataSource (Sqlite DB)
- Create Data Asset (Sqlite Table)
- Create Batch Definition (Sqlite)
- Create DataSource (Pandas)
- Create Data Asset (Pandas)
- Create Batch Definition (Pandas)
- Create Expectations 
- Create Expectations Suite and Add expectations
- Create Validation Definition (Expectation Suite, Batch Definition) - Sqlite
- Create Validation Definition (Expectation Suite, Batch Definition) - Pandas
- Create Checkpoint Sqlite (Validation Definitions)
- Create Checkpoint Pandas (Validation Definitions)
- Define Run Checkpoint (Sqlite)
- Define Batch Parameters (Pandas)
- Run Checkpoint(Pandas)
- Print Result

### Import Libraries

In [1]:
import great_expectations as gx
import pandas as pd


### Creating GX Context

In [2]:
context = gx.get_context(mode="ephemeral")


### Creating Datasource  (Sqlite)

In [3]:
datasource_name_sqlite = "sqlite_datasource"
data_source_sqlite = context.data_sources.add_sqlite(
    name=datasource_name_sqlite,
    connection_string="sqlite:///temperature.db"
)


### Creating Data Asset (Sqlite)

In [4]:
data_asset_name_sqlite = 'temperature_reading'
data_asset_sqlite = data_source_sqlite.add_table_asset(name = data_asset_name_sqlite, table_name ='temperature_reading')


### Creating Batch Definition (Sqlite)

In [5]:
batch_definition_sqlite = data_asset_sqlite.add_batch_definition("full_table")


### Creating Datasource (Pandas)

In [6]:
data_source_name_pandas ='temperature_data'
data_source_pandas = context.data_sources.add_pandas(name=data_source_name_pandas)


### Creating Data Asset (Pandas)

In [7]:
data_asset_name_pandas = 'temperature_entity_asset'
data_asset_pandas = data_source_pandas.add_dataframe_asset(name = data_asset_name_pandas)


### Creating Batch Definition (Pandas)

In [8]:
batch_definition_name_pandas ='full_batch'
batch_definition_pandas = data_asset_pandas.add_batch_definition_whole_dataframe(batch_definition_name_pandas)


### Creating Expectations

In [9]:
expectation_temp = gx.expectations.ExpectColumnValuesToBeBetween(column="temperature", 
                                                            max_value=45, min_value=5)


In [10]:
expectation_city = gx.expectations.ExpectColumnDistinctValuesToBeInSet(column="city", 
                                                            value_set=["Mumbai", "Delhi"])


### Creating Expectation Suite and Adding expectations

In [11]:
expectation_suite_name = "temperature_data_suite"
expectation_suite_ref = gx.ExpectationSuite(name=expectation_suite_name)
expectation_suite = context.suites.add(expectation_suite_ref)


In [12]:
expectation_suite.add_expectation(expectation_temp)
expectation_suite.add_expectation(expectation_city)


ExpectColumnDistinctValuesToBeInSet(id='f2749a07-1ea0-4dcf-9c53-6545672700d1', meta=None, notes=None, result_format=<ResultFormat.BASIC: 'BASIC'>, description=None, catch_exceptions=False, rendered_content=None, windows=None, batch_id=None, column='city', row_condition=None, condition_parser=None, value_set=['Mumbai', 'Delhi'])

### Creating Validation Defintions (Sqlite)

In [13]:
validation_def_name_sqlite = "temperature_data_validation_sqlite"
validation_definition_ref_sqlite = gx.ValidationDefinition( data=batch_definition_sqlite,
                                             suite=expectation_suite,
                                             name=validation_def_name_sqlite)


In [14]:
validation_definition_sqlite = context.validation_definitions.add(validation_definition_ref_sqlite)


### Creating Validation Definition (Pandas)

In [15]:
validation_def_name_pandas = "temperature_data_validation_pandas"
validation_definition_ref_pandas = gx.ValidationDefinition( data=batch_definition_pandas,
                                             suite=expectation_suite,
                                             name=validation_def_name_pandas)


In [16]:
validation_definition_pandas = context.validation_definitions.add(validation_definition_ref_pandas)


### Creating Checkpoint (Sqlite)

In [17]:
checkpoint_name_sqlite = "temperature_run_checkpoint_sqlite"
checkpoint_to_add_sqlite = gx.Checkpoint(name=checkpoint_name_sqlite,                                 
                                  validation_definitions=[validation_definition_sqlite ],
                                  actions=[],
                                  result_format="BASIC"
)


In [18]:
checkpoint_sqlite = context.checkpoints.add(checkpoint_to_add_sqlite)


### Creating Checkpoint(Pandas)

In [19]:
checkpoint_name_pandas= "temperature_run_checkpoint_pandas"
checkpoint_to_add_pandas = gx.Checkpoint(name=checkpoint_name_pandas,                                 
                                  validation_definitions=[validation_definition_pandas ],
                                  actions=[],
                                  result_format="BASIC"
)


In [20]:
checkpoint_pandas = context.checkpoints.add(checkpoint_to_add_pandas)


### Running Checkpoint (Sqlite)

In [21]:
validation_result_sqlite = checkpoint_sqlite.run()


Calculating Metrics:   0%|          | 0/12 [00:00<?, ?it/s]

### Creating Batch Parameter for Pandas Checkpoint

In [22]:
data_df = pd.read_csv('temperature.csv')
data_df.columns = ['date','city','temperature']
batch_parameters = {"dataframe": data_df}


### Running Checkpoint (Pandas)

In [23]:
validation_result_pandas = checkpoint_pandas.run(batch_parameters = batch_parameters)


Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

### Printing Result

In [26]:
print()
print("#######################################################################################################")
print('------------------------------Sqlite Validation Result-------------------------------------------------')
print("#######################################################################################################")
print()
print(validation_result_sqlite)
print()
print("#######################################################################################################")
print('------------------------------Pandas Validation Result-------------------------------------------------')
print("#######################################################################################################")
print()
print(validation_result_pandas)



#######################################################################################################
------------------------------Sqlite Validation Result-------------------------------------------------
#######################################################################################################
run_id={"run_name": null, "run_time": "2025-06-23T17:38:57.260745+06:00"} run_results={ValidationResultIdentifier::temperature_data_suite/__none__/20250623T113857.260745Z/sqlite_datasource-temperature_reading: {
  "success": true,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_be_between",
        "kwargs": {
          "batch_id": "sqlite_datasource-temperature_reading",
          "column": "temperature",
          "min_value": 5.0,
          "max_value": 45.0
        },
        "meta": {},
        "id": "9b8cd462-48d4-44bd-a8b2-e4c78f91b526"
      },
      "result": {
        "element_count": 62,
        "unexp